# Section 4.4 - Stable Marrige
### *Given lists of preferences as input, can we find a stable marriage? Can we even guarantee a stable marriage will exist for any set of preferences?*

----------

#### **Definition 4.13**. A bijection $f : M -> W$ is called *stable* if there is no pair $(m, w)$ $m \in M$ and $w \in W$ such that the following two conditions hold:

1. $f(m) \neq w$
2. The pair $(m, w)$ mutually prefer each other over their assigned matches. Both
   
   $rank_m(w) < rank_m(f(m))$ and $rank_w(m) < rank_w(f^{-1}(w))$


-----------

##### Note: This is not the *best* or *optimal* way to pair elements, only a pairing that satisfies the above. Many people could be unhappy and it be a stable match 

After all the proposals the suiteds need to hold on to their top preference from their proposal list

    while there are suitors

        for every suitor
    
            propose top pref (suited)
        
            suited updates current proposals with suitor

        for every suited

            hold top pref

            reject the others -> the rejected proposals go back to the set of avaliable suiters

        for every suiter in avalialbe suiters

            # if they are still in avaliable suiters they have been rejected by their top pref so,
            update top prefrence to pref + 1

# StableMatching

In [137]:
from __future__ import annotations

from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Dict, List, Optional, Set, Iterable


class Person(ABC):
    """Base class for entities to be matched in the stable matching program."""

    def __init__(self, id: int, preferences: Dict[int, int]) -> None:
        self.id = id
        # preferences: partner_id -> rank (smaller = better)
        self.preferences = preferences

    @abstractmethod
    def update(self):
        ...

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({vars(self)!r})"


class Man(Person):
    """Suitor in deferred acceptance."""

    def __init__(self, id: int, preferences: Dict[int, int]) -> None:
        super().__init__(id, preferences)
        self.proposal_count: int = 0
        # Precompute preference order once: list of woman_ids from best to worst
        self.preference_order: List[int] = sorted(self.preferences, key=self.preferences.get)

    def next_choice(self) -> int:
        """Return the next woman id to propose to, and advance pointer."""
        if self.proposal_count >= len(self.preference_order):
            raise RuntimeError(f"Man {self.id} has no one left to propose to.")
        w_id = self.preference_order[self.proposal_count]
        self.proposal_count += 1
        return w_id

    def update(self) -> None:
        """Optional hook; not needed in basic DA."""
        return None


class Woman(Person):
    """Suited in deferred acceptance."""

    def __init__(self, id: int, preferences: Dict[int, int]) -> None:
        super().__init__(id, preferences)
        self.current_proposals: Set[int] = set()
        self.match: Optional[int] = None  # man_id currently held (tentative)

    def add_proposal(self, proposer_id: int) -> None:
        self.current_proposals.add(proposer_id)

    def prefers(self, m1: int, m2: Optional[int]) -> bool:
        """Return True if woman prefers m1 over m2 (where m2 may be None)."""
        if m2 is None:
            return True
        return self.preferences[m1] < self.preferences[m2]

    def update(self) -> Set[int]:
        """
        Consider current proposals plus existing match; keep best, reject the rest.
        Returns the set of rejected men ids.
        """
        candidates = set(self.current_proposals)
        if self.match is not None:
            candidates.add(self.match)

        if not candidates:
            self.current_proposals.clear()
            return set()

        best = min(candidates, key=lambda m_id: self.preferences[m_id])

        rejected = candidates - {best}
        self.match = best
        self.current_proposals.clear()
        return rejected


class StableMatching:
    """
    Gale–Shapley deferred acceptance (men propose).
    After build(), you get a stable matching that is man-optimal.
    """

    def __init__(self, men: Iterable[Man], women: Iterable[Woman]) -> None:
        self.men: Dict[int, Man] = {m.id: m for m in men}
        self.women: Dict[int, Woman] = {w.id: w for w in women}

        # Will be populated after build():
        self.match_m_to_w: Dict[int, int] = {}  # man_id -> woman_id
        self.match_w_to_m: Dict[int, int] = {}  # woman_id -> man_id

    def build(self) -> None:
        """Run the deferred acceptance algorithm."""
        free_men: Set[int] = set(self.men.keys())

        while free_men:
            m_id = free_men.pop()
            m = self.men[m_id]

            # Man proposes to next choice
            w_id = m.next_choice()
            w = self.women[w_id]
            w.add_proposal(m_id)

            # Woman processes proposals (and maybe dumps someone)
            rejected = w.update()

            # Any rejected men become free again (unless they have no options left)
            for rej_id in rejected:
                if rej_id in self.men:
                    # If they still have someone left to propose to, keep them in the loop
                    if self.men[rej_id].proposal_count < len(self.men[rej_id].preference_order):
                        free_men.add(rej_id)
                    else:
                        # No one left to propose to -> unmatched (shouldn't happen in classic equal-size complete prefs)
                        pass

            # The current proposer might still be rejected (if he wasn't best)
            # The above rejected set already covers him if applicable.

        # Build final match dictionaries
        self.match_w_to_m = {w_id: w.match for w_id, w in self.women.items() if w.match is not None}
        self.match_m_to_w = {m_id: w_id for w_id, m_id in self.match_w_to_m.items()}

    def __call__(self, man_id: int) -> Optional[int]:
        """Given a man_id, return matched woman_id (or None if unmatched)."""
        return self.match_m_to_w.get(man_id)

    def pairs(self) -> List[tuple[int, int]]:
        """Return list of (man_id, woman_id) pairs."""
        return sorted(self.match_m_to_w.items())

    def is_bijection(self) -> bool:
        """
        True iff the current matching is a bijection M -> W:
        - every man matched to exactly one woman
        - every woman matched to exactly one man
        - sizes match |M| == |W| == number of pairs
        """
        if len(self.men) != len(self.women):
            return False

        if len(self.match_m_to_w) != len(self.men):
            return False

        # Check uniqueness on the W side
        used_w = set(self.match_m_to_w.values())
        if len(used_w) != len(self.men):
            return False

        # Consistency with inverse mapping
        if len(self.match_w_to_m) != len(self.women):
            return False

        for m_id, w_id in self.match_m_to_w.items():
            if self.match_w_to_m.get(w_id) != m_id:
                return False

        return True

    def blocking_pairs(self) -> List[Tuple[int, int]]:
        """
        Return a list of all blocking pairs (m_id, w_id) under Definition 4.13.
        Requires that build() has been run (or match maps set).
        """
        blocks: List[Tuple[int, int]] = []

        # If not everyone matched, definition still makes sense, but you need conventions.
        # Here we assume a perfect matching (bijection). If not, you can still run it,
        # but ranks for "None" need a rule (e.g., treat unmatched as worst).
        if not self.match_m_to_w or not self.match_w_to_m:
            raise RuntimeError("No matching found. Run build() first.")

        for m_id, m in self.men.items():
            current_w: Optional[int] = self.match_m_to_w.get(m_id)
            if current_w is None:
                continue  # or treat as worst match if you want partial match support

            for w_id, w in self.women.items():
                if w_id == current_w:
                    continue  # condition (1): f(m) != w

                current_m_for_w: Optional[int] = self.match_w_to_m.get(w_id)
                if current_m_for_w is None:
                    continue  # same comment re: partial matching

                # Condition (2): mutual preference over assigned matches
                # smaller rank value = more preferred
                m_prefers = m.preferences[w_id] < m.preferences[current_w]
                w_prefers = w.preferences[m_id] < w.preferences[current_m_for_w]

                if m_prefers and w_prefers:
                    blocks.append((m_id, w_id))

        return blocks

    def is_stable(self) -> bool:
        """True iff there are no blocking pairs (Definition 4.13)."""
        return len(self.blocking_pairs()) == 0



In [135]:
men = [
    Man(1, {1: 0, 2: 1, 3: 2}),
    Man(2, {2: 0, 1: 1, 3: 2}),
    Man(3, {2: 0, 3: 1, 1: 2}),
]

women = [
    Woman(1, {2: 0, 1: 1, 3: 2}),
    Woman(2, {1: 0, 2: 1, 3: 2}),
    Woman(3, {1: 0, 3: 1, 2: 2}),
]

sm = StableMatching(men, women)
sm.build()
print(sm.pairs())     # [(1, 2), (2, 1), (3, 3)] for example
print(sm(1))          # woman matched to man 1
print(sm.is_bijection())
print(sm.is_stable())

[(1, 1), (2, 2), (3, 3)]
1
True
True
